## Loss learning evaluation

### setup

In [ ]:
import numpy as np
import math
import random
import os
import os.path
import torch
import sys
import copy
import pickle
import importlib
import torch.nn as nn
import torch
import time
import functorch
import ifcopenshell
from numpy.random import default_rng

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt
from chamferdist import ChamferDistance
from pathlib import Path
import open3d as o3d
import pytorch3d.transforms as trnsfrm

from src.dataset import *
from src.elements import *
from src.chamfer import get_cloud_chamfer_loss_tensor
from src.plots import plot_error_graph, plot_single_parameter_error
from src.meta import get_rand_rotations
from src.visualisation import vis_ifc_and_cloud
from tqdm.notebook import tqdm

In [ ]:
random.seed = 42
rng = default_rng()

In [ ]:
path = Path("output/")
savepath = Path("meta/outputs/")
cuda = torch.device("cuda")

In [ ]:
train_transforms = transforms.Compose(
    [
        Normalize(),
        #                    RandomNoise(),
        ToTensor(),
    ]
)

In [ ]:
# load data and model
BASE_DIR = os.path.dirname(os.path.abspath("industrial-facility-relationships/"))
BASE_DIR = os.path.join(BASE_DIR, "pointnet2")
ROOT_DIR = BASE_DIR
sys.path.append(os.path.join(ROOT_DIR, "models"))


path = Path("output/")
ext = ".pcd"

cat = "elbow"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_normals = False

test_ds = PointCloudData(
    path, valid=True, folder="test", category=cat, transform=train_transforms
)
targets = test_ds.targets

testDataLoader = torch.utils.data.DataLoader(dataset=test_ds, batch_size=128)
test_criterion = nn.MSELoss()

model_name = "pointnet2_meta_ssg"
# model_path = Path("pointnet2/log/classification/pointnet2_meta_ssg/")
model = importlib.import_module(model_name)
checkpoint_path = Path("pointnet2/log/meta/checkpoints/")


fcn_model = importlib.import_module("fcn")
fcn_predictor = fcn_model.get_model(3)

predictor = model.get_model(normal_channel=False)
if device != "cpu":
    predictor = predictor.cuda()
    fcn_predictor = fcn_predictor.cuda()

checkpoint = torch.load(checkpoint_path / "best_model.pth")
fcn_checkpoint = torch.load(checkpoint_path / "best_model_fcn.pth")
predictor.load_state_dict(checkpoint["model_state_dict"])
fcn_predictor.load_state_dict(fcn_checkpoint["model_state_dict"])

output_dir = Path("meta/")

### Inference

In [ ]:
def model_inference(model, loader, device, calculate_score=False, loss_type="chamfer"):
    predictor = model.eval()
    predictions_list, pcd_list, transformed_pcd_list, id_list, chamfer_dist_list = (
        [],
        [],
        [],
        [],
        [],
    )
    with torch.no_grad():
        for j, data in tqdm(enumerate(loader), total=len(loader)):
            points, ids = data["pointcloud"].to(device).float(), data["id"].to(device)

            # perform rotation
            rand_rot = get_rand_rotations(points.shape[0], device=device, scale=0.05)
            trans = trnsfrm.Rotate(rand_rot)
            points_transformed = trans.transform_points(points)

            points = points.transpose(2, 1)
            points_transformed = points_transformed.transpose(2, 1)

            # get predictions and concatenate to single tensor
            pred, _ = predictor(points)
            pred_trans, _ = predictor(points_transformed)
            pred_combined = torch.cat([pred, pred_trans], 1)
            predicted_rotation = fcn_predictor(pred_combined)
            rot_mat = trnsfrm.euler_angles_to_matrix(predicted_rotation, convention="XYZ")

            # rotate original cloud by predicted rotation
            # print(predicted_rotation.shape)
#             predicted_rotation = torch.reshape(
#                 predicted_rotation, (predicted_rotation.shape[0], 3, 3)
#             )
            trans = trnsfrm.Rotate(rot_mat)
            points_t = points.transpose(2, 1)
            points_transformed2 = trans.transform_points(points_t)
            points_transformed2 = points_transformed2.transpose(2, 1)

            if loss_type == "chamfer":
                loss = get_cloud_chamfer_loss_tensor(
                    points_transformed2,
                    points_transformed,
                    separate_directions=False,
                    reduction=None,
                )
            else:
                loss = torch.sum(
                    torch.square(points_transformed2 - points_transformed), dim=(1, 2)
                )
                loss = loss.mean()
            # print("chamfer_loss: ", chamfer_loss)

            predicted_rotation, points, transformed_points, ids, loss = (
                predicted_rotation.to(torch.device("cpu")),
                points_transformed2.to(torch.device("cpu")),
                points_transformed.to(torch.device("cpu")),
                data["id"].to(torch.device("cpu")),
                loss.to(torch.device("cpu")),
            )

            for i, pr in enumerate(predicted_rotation):
                predictions_list.append(pr.numpy())
                pcd_list.append(points[i].numpy())
                transformed_pcd_list.append(transformed_points[i].numpy())
                id_list.append(ids[i].numpy())
                chamfer_dist_list.append(loss[i].numpy())

        return (
            predictions_list,
            pcd_list,
            transformed_pcd_list,
            id_list,
            chamfer_dist_list,
        )

In [ ]:
(
    predictions_list,
    pcd_list,
    transformed_pcd_list,
    id_list,
    chamfer_dist_list,
) = model_inference(predictor.eval(), testDataLoader, device, "direct")

In [ ]:
print(len(chamfer_dist_list), chamfer_dist_list[0].shape)

In [ ]:
plot_error_graph(chamfer_dist_list, "Predicted chamfer loss", max_val=1000)

In [ ]:
# plot_error_graph(chamfer_dist_list, "Actual chamfer loss", max_val=1000)

### visualisation

In [ ]:
print(type(pcd_list[i][0][0].astype("float64")))

In [ ]:
blueprint = "data/sample.ifc"
ifc = setup_ifc_file(blueprint)
limit = 10
vis = []

for i in range(limit):
    vis.append(vis_ifc_and_cloud(ifc, [pcd_list[i].transpose().astype("float64"), transformed_pcd_list[i].transpose().astype("float64")]))

vis[0]

In [ ]:
chamfer_dist_list_nested = [[i] for i in chamfer_dist_list]
predictions_list_nested = [[i] for i in predictions_list]
plot_single_parameter_error(
    chamfer_dist_list_nested,
    predictions_list_nested,
    0,
    "dimension",
    "chamfer distance",
)

In [ ]:
print(chamfer_dist_list[:10])
print(predictions_list[:10])

In [ ]:
cld = o3d.geometry.PointCloud()

for i, p in enumerate(pcd_list[:10]):
    points = o3d.utility.Vector3dVector(p.transpose())
    cld.points = points
    o3d.io.write_point_cloud(str(savepath / (str(i) + ".ply")), cld)


for i, p in enumerate(transformed_pcd_list[:10]):
    points = o3d.utility.Vector3dVector(p.transpose())
    cld.points = points
    o3d.io.write_point_cloud(str(savepath / (str(i) + "_t.ply")), cld)

In [ ]:
rand_euler = np.random.rand(2, 3) * 2 * np.pi
print(rand_euler)

# scale down the rotation to be closer to the original
den = np.random.rand(2, 1)
print(den)
rand_euler = rand_euler * den * den
print(rand_euler)
rot_mat = trnsfrm.euler_angles_to_matrix(torch.Tensor(rand_euler), convention="XYZ")
# den = np.random.rand(1)

In [ ]:
trans = trnsfrm.Rotate(rot_mat)
points = torch.Tensor([pcd_list[0].transpose()])
print(points.shape, rot_mat.shape)
points_transformed = trans.transform_points(points)
points = points.detach().numpy()[0]
points_transformed = points_transformed.detach().numpy()[0]

p = o3d.utility.Vector3dVector(points)
cld.points = p
o3d.io.write_point_cloud(str(savepath / ("t_t.ply")), cld)

p = o3d.utility.Vector3dVector(points_transformed)
cld.points = p
o3d.io.write_point_cloud(str(savepath / ("t.ply")), cld)